In [1]:
!pip install -r requirements.txt

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
%reload_ext autoreload

In [2]:
#lst = list(await wiki_bench.client.client.models.list())[0][1]
#for l in lst:
#    print(l)

NameError: name 'wiki_bench' is not defined

In [1]:
import sys

sys.path.append("src")
from wiki_bench import WikiBench
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()
#wiki_bench = WikiBench(url, key, model_name='RefalMachine/RuadaptQwen2.5-7B-Lite-Beta', pre_load=True)

resource module not available on Windows


In [2]:
import logging
import torch
from sentence_transformers import SentenceTransformer

logging.getLogger("sentence_transformers.SentenceTransformer").setLevel(logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer("sergeyzh/BERTA").to(device)

In [3]:
import asyncio
import json
from datetime import datetime, UTC
import aiofiles
import os


async def run_bench_for_model(url: str,
                              api_key: str,
                              model_name: str,
                              device = None,
                              encoder = None,
                              out_dir: str = "results",
                              reference_mode: int = 0,
                              neighbor_count = 0,
                              description_mode = 0,
                              mode = 1
                             ):

    file_path = f"{out_dir}/{model_name.replace('/', '_')}.jsonl"

    os.makedirs(out_dir, exist_ok=True)

    bench = WikiBench(url, api_key, model_name=model_name, device=device, encoder=encoder, pre_load=True)
    
    async with aiofiles.open(file_path, "a", encoding="utf‑8") as f:
        
        # ---------- запросы ----------
        async for article_name, item, annotation in bench.query_stream(reference_mode=reference_mode):
            await f.write(_as_jsonl(model_name, article_name, "query", item, annotation, False))
        print(f"[{model_name}] — запросы готовы")
        
        # ---------- планы статей ----------
        async for article_name, item in bench.outline_stream(
            neighbor_count=neighbor_count, 
            description_mode=description_mode, 
            mode=mode
        ):
            await f.write(_as_jsonl(model_name, article_name, "outline", item, '', False))
        print(f"[{model_name}] — планы готовы")

        # ---------- секции статей ----------
        async for article_name, item in bench.sections_stream():
            await f.write(_as_jsonl(model_name, article_name, "section", item, '', True))
        print(f"[{model_name}] — секции готовы")

    print(f"[{model_name}] — готово. Результаты: {file_path}")
    
def serialize(d):
    return [[[k, n], v] for (k, n), v in d.items()]

def _as_jsonl(model, article_name, phase, data, annotation='', needs_ser=False):
    if needs_ser:
        data = serialize(data)
    entry = {
        "ts": datetime.now(UTC).isoformat(timespec="seconds"),
        "model": model,
        "article_name": article_name,
        "phase": phase,
        "annotation": annotation,
        "data": data,
    }
    return json.dumps(entry, ensure_ascii=False) + "\n"


In [4]:
from openai_utils import AsyncList

async def main():
    models = AsyncList()
    model_names = [
        'RefalMachine/RuadaptQwen3-32B-Instruct-v2',
        'Qwen3-235B-A22B',
        'DeepSeek V3',
        'tpro',
        'RefalMachine/RuadaptQwen2.5-7B-Garant-v2',
        'yagpt5lite'
    ]
    for name in model_names:
        models.append(run_bench_for_model(url, key, name, device, encoder))

    await models.complete_couroutines(batch_size=6)
    models = await models.to_list()

In [5]:
await main()

[yagpt5lite] — запросы готовы
[RefalMachine/RuadaptQwen2.5-7B-Garant-v2] — запросы готовы
[yagpt5lite] — планы готовы
[Qwen3-235B-A22B] — запросы готовы
[RefalMachine/RuadaptQwen2.5-7B-Garant-v2] — планы готовы
[DeepSeek V3] — запросы готовы
[Qwen3-235B-A22B] — планы готовы
[DeepSeek V3] — планы готовы
[yagpt5lite] — секции готовы
[yagpt5lite] — готово. Результаты: results/yagpt5lite.jsonl
[RefalMachine/RuadaptQwen2.5-7B-Garant-v2] — секции готовы
[RefalMachine/RuadaptQwen2.5-7B-Garant-v2] — готово. Результаты: results/RefalMachine_RuadaptQwen2.5-7B-Garant-v2.jsonl
[RefalMachine/RuadaptQwen3-32B-Instruct-v2] — запросы готовы
[tpro] — запросы готовы
[RefalMachine/RuadaptQwen3-32B-Instruct-v2] — планы готовы
[tpro] — планы готовы
[Qwen3-235B-A22B] — секции готовы
[Qwen3-235B-A22B] — готово. Результаты: results/Qwen3-235B-A22B.jsonl
[DeepSeek V3] — секции готовы
[DeepSeek V3] — готово. Результаты: results/DeepSeek V3.jsonl


APIConnectionError: Connection error.